In [1]:
import csv
import pandas as pd
import numpy as np
import os

In [2]:
listas = []
for i in range(2):
    listas.append(raw_input("Ingresar ruta a lista %s: " % (i+1)))
    if not(os.path.isfile(listas[i])):
        print "Path a Archivo no es válido"
        exit

Ingresar ruta a lista 1: /home/juan/Documentos/UNLu/taller_libre_1/2016/practica/tp2/punto7/my_retrieval/normalized_ranking.res
Ingresar ruta a lista 2: /home/juan/Documentos/UNLu/taller_libre_1/2016/practica/tp2/punto7/terrier_retrieval/TF_IDF_0.res


In [3]:
data = []
for i in range(2):
    data.append(pd.read_csv(listas[i], sep=" ", header = None, names=["Id query", "Q", "Id doc", "Ranking", "Similtud", "Modelo"], index_col=["Id query","Id doc"]))
    modelo = data[i]['Modelo'].iloc[0]
    data[i].drop(['Q','Modelo'], axis=1, inplace=True)
    data[i].columns = ["Ranking - "+modelo, "Similtud - "+modelo]
data[1].head()

Ranking - TF_IDF  Similtud - TF_IDF
Id query Id doc                                     
1        122                    0           2.286827
         206                    1           2.251620
         328                    2           2.235622
         50                     3           2.226903
         171                    4           2.221991

In [4]:
df = pd.concat([data[0], data[1]],axis=1,join="outer")

# Ordeno por iDQUERY y luego por ranking del primero de los dos modelos
df = df.reset_index()
df = df.sort_values(["Id query",df.columns[2]])
df = df.set_index(["Id query", "Id doc"])

df.to_csv("resultados_queries.csv")
df.head()


Ranking - JCAR_2016  Similtud - JCAR_2016  Ranking - TF_IDF  \
Id query Id doc                                                                
1        206                     0.0              0.017707               1.0   
         122                     1.0              0.017401               0.0   
         576                     2.0              0.013259               5.0   
         327                     3.0              0.010090               9.0   
         587                     4.0              0.009448              13.0   

                 Similtud - TF_IDF  
Id query Id doc                     
1        206              2.251620  
         122              2.286827  
         576              2.204487  
         327              2.145982  
         587              2.119273

In [7]:
# Elimino columnas que contienen valor de similtud (no me interesa para clcular spearman)
spearman = df.drop(df.columns[[1,3]], axis=1)
spearman.head(15)

Ranking - JCAR_2016  Ranking - TF_IDF
Id query Id doc                                       
1        206                     0.0               1.0
         122                     1.0               0.0
         576                     2.0               5.0
         327                     3.0               9.0
         587                     4.0              13.0
         393                     5.0              35.0
         413                     6.0               6.0
         284                     7.0              50.0
         328                     8.0               2.0
         405                     9.0              11.0
         50                     10.0               3.0
         504                    11.0              56.0
         171                    12.0               4.0
         569                    13.0              36.0
         106                    14.0              46.0

In [46]:
# Remuevo valores NaN (cuando un modelo devuelve un documento pero el otro no)
fillValues = spearman.max(level="Id query")+1
spearman = spearman.fillna(value=fillValues)
spearman.isnull().sum()

Ranking - JCAR_2016    0
Ranking - TF_IDF       0
dtype: int64

In [47]:
def coeficienteSpearman(df):
    n = len(df)
    df['d'] = df[df.columns[0]] - df[df.columns[1]]
    df['d_cuadrado'] = np.power(df['d'],2)
    sum_d_cuadrado = sum(df['d_cuadrado']) 
    resultado = 1 - ( (6.0 * sum_d_cuadrado) / (n * ( np.power(n,2.0) - 1.0 ) ) )
    # WARNING: NO ANDA BIEN
    #if n > 20:
    #    resultado = resultado / np.sqrt( (1.0 - np.power( resultado,2.0 )) / (n - 2.0) )
    return resultado 


In [48]:
# Creo el data frame resultado en donde muestro el coeficiente obtenido para cada query
spearman = spearman.reset_index()
df = pd.DataFrame(index=np.arange(min(spearman['Id query']), max(spearman['Id query'])+1), columns=['Coeficiente (n=10)', 'Coeficiente (n=25)', 'Coeficiente (n=50)'], dtype = 'float64')
df.index.name = "Id query"

# A cada index le doy su coeficiente para los primeros 'n' resultados
for i in df.index:
    
    # creo dataframe 'query' con los datos del query con id 'i'
    query = spearman[spearman['Id query'] == i]
    query = query.drop("Id query", axis=1)
    query = query.set_index(["Id doc"])
    # Calculo coeficiente
    df.loc[i,"Coeficiente (n=10)"] = coeficienteSpearman(query.head(n=10)) 
    df.loc[i,"Coeficiente (n=25)"] = coeficienteSpearman(query.head(n=25)) 
    df.loc[i,"Coeficiente (n=50)"] = coeficienteSpearman(query.head(n=50)) 
    
# Restauro index 
spearman = spearman.set_index(["Id query", "Id doc"])

df.head()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Coeficiente (n=10),Coeficiente (n=25),Coeficiente (n=50)
Id query,,,
1,-16.678788,-2.682692,-0.333445
2,-943.866667,-170.877692,-55.511020
3,-754.806061,-70.694615,-17.436735
4,-28.181818,-1.592308,0.565102
5,1.000000,1.000000,1.000000


In [49]:
df.to_csv("coeficiente_spearman.csv")